In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import corner
import emcee
import george

In [ ]:
from gpr_mcmc import *

## Simulate data

In [ ]:
X = np.linspace(start=0, stop=10, num=1000)
y = np.squeeze(X * np.sin(X))

In [ ]:
#rng = np.random.RandomState(1)
training_indices = np.random.choice(np.arange(y.size), size=10, replace=False)
X_train, y_train = X[training_indices], y[training_indices]

In [ ]:
noise = np.random.normal(loc=0.0, scale=0.75, size=y_train.shape)
y_train_noise = y_train + noise

In [ ]:
plt.plot(X, y, label=r"$f(x) = x \sin(x)$", linestyle="dotted")
plt.errorbar(X_train, y_train_noise, np.abs(noise), label='train (noise)', fmt='oC1')
plt.legend()

## Fit GPR using MCMC

In [ ]:
initial_log, priors_log = get_initial_ln_prior(y_train_noise)

In [ ]:
sampler = run_mcmc(X_train, y_train_noise, noise,  priors_log, initial_log, nwalkers=100, nsteps=1000)

In [ ]:
plot_walkers(sampler)

In [ ]:
samples_y_log, log_L = plot_scatter_matrix(sampler, thin=1)

## Evaluate model

In [ ]:
theta_results = get_theta(samples_y_log)

In [ ]:
gp_best_fit = build_gp(np.log(theta_results), X_train, y_train_noise, noise)

In [ ]:
mu_fit, cov_fit = gp_best_fit.predict(y_train_noise, X_train)
sig_fit = np.sqrt(np.diag(cov_fit))

In [ ]:
mu_pred, cov_pred = gp_best_fit.predict(y_train_noise, X)
sig_pred = np.sqrt(np.diag(cov_pred))

In [ ]:
fig, axs = plt.subplots(2, 1)
axs[0].plot(X, y, label=r"$f(x) = x \sin(x)$", linestyle="dotted")
axs[0].errorbar(X_train, y_train_noise, yerr=np.abs(noise), label='train (noise)', fmt='oC1')
axs[0].plot(X, mu_pred, color='C1')
axs[0].fill_between(X, mu_pred + sig_pred, mu_pred - sig_pred, color='C1', alpha=0.3)
axs[1].errorbar(X_train, y_train_noise-mu_fit, yerr=np.abs(noise), fmt='ok')

[Source - Charles Cadieux](https://exoplanetes.umontreal.ca/en/team-member/charles-cadieux/)